# Cofactor swap targets

The conversion of tryptamine to 4-hydroxy-tryptamine uses NADPH as a cofactor, and therefore a cofactor swap analysis was performed in order to determine whether increasing the pool of available NADPH by cofactor swapping in any reactions in the model could increase psilocybin production. 

In [1]:
%run 01-model.ipynb

from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

'' is not a valid SBML 'SId'.


In [2]:
model.solver = "glpk" # what is this?

In [3]:
biomass = model.reactions.DRAIN_Biomass

In [4]:
biomass.lower_bound = 0.1 # OBS denne skal måske ikke være 0,1 tjek tutorial notebooken og forstå det


In [5]:
demand = model.reactions.DM_psi

The demand reaction for the "target" (psilocybin) is defined in "01-model.ipynb"

In [6]:
model.objective = demand

In [7]:
product_yield = product_yield(demand, model.reactions.BOUNDARY_GLCe)

In [8]:
CofactorSwapOptimization?

Init signature:
CofactorSwapOptimization(
    cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)
Docstring:     
Optimize co-factor swapping

As suggested in [1]_, flux through a given reaction can sometimes be optimized by swapping complementary
co-factor. This class implements a search for reactions when swapped improve the given objective. Briefly,
the approach is to

- find reactions that have all the targeted co-factor pairs e.g. (nad_c -> nadp_c, nadh_c -> nadph_c)

- add reactions that have the co-factors swapped and then by a search algorithm switching one off in favor of the
  other

The implementation here differs from that in [1]_ in that we use a general purpose search algorithm rather than
formulating the search as a mixed integer linear programming problem.

References
----------
.. [1] King, Zachary A., and Adam M. Feist. "Optimizing Cofactor Specificity of Oxidoreductase Enzym

Our model has different syntax for the default cofactors, and therefore the parameter cofactor_id_swaps must be added to the CofactorSwapOptimization function input with the correct syntax. 

In [9]:
swap_opt_NAD = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['NAD', 'NADH'], ['NADP', 'NADPH']))

In [10]:
%%time
result_NAD = swap_opt_NAD.run(max_size=5)

Starting optimization at Sun, 29 Nov 2020 21:17:06


HBox()

Finished after 00:01:26
CPU times: user 2min 41s, sys: 0 ns, total: 2min 41s
Wall time: 2min 41s


In [11]:
result_NAD

,index,targets,fitness


Cofactor swapping NADH for NADPH has no effect on the production flux of DM_psi. This may be due to an NADH requirement for reactions in the precursor pathways for tryptophan production. Next we will try swapping FADH2p for NADPH, to see if this could increase the pool of NADPH and improve psilocybin production. The model doens't contain FADHc so therefore FADH2p is swapped instead.

In [25]:
swap_opt_FADp = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['FADp', 'FADH2p'], ['NADP', 'NADPH']))

In [26]:
%%time
result_FADp = swap_opt_FADp.run(max_size=5)

Starting optimization at Sun, 29 Nov 2020 21:38:30


HBox()

Finished after 00:01:24
CPU times: user 2min 33s, sys: 53.2 ms, total: 2min 33s
Wall time: 2min 33s


In [27]:
result_FADp

,index,targets,fitness
0,0,"(r162,)",0.550228
1,1,"(r160,)",0.550228


The CofactorSwapOptimization revealed r162 and r160 as target reactions for cofactor swapping FADPHp for NADPH, for increasing psilocybin production.

In [28]:
model.reactions.r162

Reaction identifier,r162
Name,Ribulose reductase
Memory address,0x07f5ac9e15e48
Stoichiometry,NADPH + RL <=> AOL + NADP NADPH + D-Ribulose <=> D-Arabitol + NADP+
GPR,1142460
Lower bound,-1000.0
Upper bound,1000.0


In [29]:
model.reactions.r160

Reaction identifier,r160
Name,Ribitol dehydrogenase
Memory address,0x07f5ac9e15e80
Stoichiometry,NADP + RIBOL <=> NADPH + RL NADP+ + Ribitol <=> NADPH + D-Ribulose
GPR,1178672 or 1170889 or 1155007
Lower bound,-1000.0
Upper bound,1000.0


ATP is the cofactor required for the last reaction in the psilocybin production pathway, psiM. Therefore the next cofactor swap will determine whether there are any reactions in which a cofactor swap of GTP for ATP could increase the pool of avaible ATP and improve psilocybin production. 

In [17]:
swap_opt_GDP_ATP = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['GDP', 'GTP'], ['ADP', 'ATP']))

In [18]:
%%time
result_GDP_ATP = swap_opt_GDP_ATP.run(max_size=5)

Starting optimization at Sun, 29 Nov 2020 21:21:59


HBox()

Finished after 00:01:20
CPU times: user 2min 20s, sys: 0 ns, total: 2min 20s
Wall time: 2min 20s


In [20]:
result_GDP_ATP

,index,targets,fitness
0,0,"(r436,)",0.926206


The CofactorSwapOptimization revealed r436 as a target reaction for swapping GTP for ATP to increase psilocybin production. 

In [21]:
model.reactions.r436

Reaction identifier,r436
Name,Adenylate kinase
Memory address,0x07f5ac9b6d6d8
Stoichiometry,AMP + ATP <=> 2.0 ADP AMP + ATP <=> 2.0 ADP
GPR,1171415 or 1144613
Lower bound,-1000.0
Upper bound,1000.0
